In [1]:
import sys
import pystac_client

In [2]:
sys.path.append("../..")

In [3]:
from cdseutils.utils import BBox

In [4]:
endpoint_url = "https://catalogue.dataspace.copernicus.eu/stac"

In [5]:
# collection = "COP-DEM"
collection = "CCM"

In [6]:
bbox = BBox(
    left=16.50,
    south=40.53,
    right=16.85,
    north=40.72,
)
bbox

BBox(left=16.5, south=40.53, right=16.85, north=40.72)

In [7]:
client = pystac_client.Client.open(endpoint_url)
assert client.conforms_to("ITEM_SEARCH")
# assert client.conforms_to("FILTER")
client.add_conforms_to("FILTER")  # silence warnings
# assert client.conforms_to("SORT")

In [8]:
client.get_conforms_to()

['https://api.stacspec.org/v1.0.0/core',
 'https://api.stacspec.org/v1.0.0/collections',
 'https://api.stacspec.org/v1.0.0/ogcapi-features',
 'https://api.stacspec.org/v1.0.0/item-search',
 'https://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
 'https://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson',
 'http://www.opengis.net/spec/cql2/1.0/conf/cql2-json',
 'https://api.stacspec.org/v1.0.*/item-search#filter']

In [9]:
collections = set(client.get_all_collections())
collections

{<CollectionClient id=CCM>,
 <CollectionClient id=CLMS>,
 <CollectionClient id=COP-DEM>,
 <CollectionClient id=ENVISAT>,
 <CollectionClient id=GLOBAL-MOSAICS>,
 <CollectionClient id=LANDSAT-5>,
 <CollectionClient id=LANDSAT-7>,
 <CollectionClient id=LANDSAT-8-ESA>,
 <CollectionClient id=LANDSAT-8>,
 <CollectionClient id=LANDSAT-9>,
 <CollectionClient id=S2GLC>,
 <CollectionClient id=SENTINEL-1-RTC>,
 <CollectionClient id=SENTINEL-1>,
 <CollectionClient id=SENTINEL-2>,
 <CollectionClient id=SENTINEL-3>,
 <CollectionClient id=SENTINEL-5P>,
 <CollectionClient id=SENTINEL-6>,
 <CollectionClient id=SMOS>,
 <CollectionClient id=SUOMI-NPP>,
 <CollectionClient id=TERRA>,
 <CollectionClient id=TERRAAQUA>}

In [10]:
assert collection in {item.id for item in collections}

In [11]:
filter = {  # noqa: A001
    "op": "=",
    "args": [
        {"property": "datasetFull"},
        "COP-DEM_GLO-90-DGED",
    ],
}

In [12]:
# sortby = [
#     {"direction": "asc", "field": "properties.gridId"},
#     {"direction": "desc", "field": "properties.dataset"},
# ]

In [13]:
dged_search = client.search(
    collections=[collection],
    filter=filter,
    bbox=bbox,
    # sortby=sortby,
    # max_items=10,
)

In [14]:
def keyfunc(item):
    return item.properties["gridId"], item.properties["dataset"]


items = sorted(dged_search.items(), key=keyfunc, reverse=True)
items

[<Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_3e420040.DEM>,
 <Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_cae55e81.DEM>,
 <Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_513411cf.DEM>,
 <Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_60ec960a.DEM>,
 <Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_d850f16f.DEM>,
 <Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_e2f8c435.DEM>]

In [15]:
[keyfunc(item) for item in items]

[('N40_E016', 'COP-DEM_GLO-90-DGED/2024_1'),
 ('N40_E016', 'COP-DEM_GLO-90-DGED/2023_1'),
 ('N40_E016', 'COP-DEM_GLO-90-DGED/2022_1'),
 ('N40_E016', 'COP-DEM_GLO-90-DGED/2021_1'),
 ('N40_E016', 'COP-DEM_GLO-90-DGED/2020_1'),
 ('N40_E016', 'COP-DEM_GLO-90-DGED/2019_1')]

In [16]:
item = items[0]

In [17]:
item

<Item id=DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_3e420040.DEM>

In [18]:
asset = item.assets["PRODUCT"]
asset

<Asset href=https://catalogue.dataspace.copernicus.eu/odata/v1/Products(d1f6d92b-c1aa-440c-88f6-bb8dfdafe191)/$value>

In [19]:
asset.href

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products(d1f6d92b-c1aa-440c-88f6-bb8dfdafe191)/$value'

In [20]:
asset.extra_fields["alternate"]["s3"]

{'href': '/eodata/CCM/COP-DEM_GLO-90-DGED/SAR_DGE_90_A407/2011/04/15/DEM1_SAR_DGE_90_20110415T164917_20140317T164937_ADS_000000_1700_3e420040',
 'storage:platform': 'CLOUDFERRO',
 'storage:region': 'waw',
 'storage:requester_pays': False,
 'storage:tier': 'Online'}